In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pickle
import sys
import re
#from pymongo import MongoClient
#import pymongo
#import dns 
import csv

modulename = "dnspython"
if modulename not in sys.modules:
    print('You have not imported the {} module'.format(modulename))
else:
    print('You have imported the {} module'.format(modulename))


In [ ]:
crawled_liste = []

class Article:
    def __init__(self, title, lead, datum, author, text, url):
        self.title = title
        self.lead = lead
        self.datum = datum
        self.author = author
        self.text = text
        self.url = url

In [ ]:
def inhalter():    
    #main
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    #segments
    title = soup.find(class_="maintitle").get_text()

    ## lead (as it is not optional to have one)
    lead = soup.find_all(class_="lead")
    if len(lead) >= 2:
        lead = lead[0].get_text()
    else:
        lead = ""

    # Datum (is not always given)
    try:
        datum = soup.find(class_="publish_date").get_text()
        datum = datum[:16]
    except AttributeError:
        datum = ""
    
    ## Author
    try:
        author = soup.find(class_=re.compile("author")).get_text()
        author = author[0:36].strip().get_text()
    except AttributeError:
        author = "Agentur"
    
    ## Text
    text = str()
    blocktext = soup.find_all("p")
    for teil in blocktext:
        texty = teil.get_text()
        texty = texty.replace('\t','')
        texty = texty.replace('\n','')
        text += " " + texty
    
    #URL
    ding_url = example
    
    crawled = Article(title, lead, datum, author, text, ding_url)
    crawled_liste.append(crawled)

In [ ]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.maximize_window()

In [ ]:
start_seite = "https://www.watson.ch/"
beispiel =["https://www.watson.ch/digital/wirtschaft/716343328-wie-gross-ist-das-risiko-eines-blackouts-das-sagt-der-bund",  "https://www.watson.ch/international/donald%20trump/623748967-us-senat-spricht-ex-praesident-trump-im-amtsenthebungsverfahren-frei"]

driver.get(start_seite)

cookies = pickle.load(open("watson_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.refresh()

In [ ]:
final_seiten = []

with open("watson_output.bin", "rb") as data:
    ueb_pages = pickle.load(data)
    
mismatch = "https://www.watson.chhtt"
for x in ueb_pages:
    if mismatch in x:
        print(x)
    else:
        final_seiten.append(x)

In [ ]:
for example in final_seiten[2955:]:
    time.sleep(3)
    driver.get(example)
    try:
        inhalter()
    except TimeoutException:
        print("TimeOut")
        pass

In [ ]:
with open('watson_art4.csv', 'w', newline='', encoding="utf8") as csvfile:
    blogwriter = csv.writer(csvfile, delimiter='|', quotechar='"')


    for article in crawled_liste:
        blogwriter.writerow( [article.title, article.lead, article.datum, article.author, article.text, article.url] )